# BANXICO

## Tasas de interés:
- **SF63528**: `Dólares` - Tipo de cambio del dólar estadounidense respecto al peso mexicano. Se utiliza para evaluar la fortaleza de la moneda nacional frente al dólar.
- **SF43718**: `TIIE_28_dias` - Tasa de Interés Interbancaria de Equilibrio a 28 días. Es una referencia para determinar las tasas de interés en operaciones financieras a corto plazo.
- **SF43936**: `TIIE_91_dias` - Tasa de Interés Interbancaria de Equilibrio a 91 días. Similar a la TIIE a 28 días, pero para un plazo mayor.
- **SF43945**: `Cetes_28_dias` - Tasa de rendimiento de los Certificados de la Tesorería (Cetes) a 28 días. Representa el rendimiento de estos instrumentos de deuda emitidos por el gobierno mexicano.
- **SF43946**: `Cetes_91_dias` - Tasa de rendimiento de los Certificados de la Tesorería (Cetes) a 91 días. Al igual que los Cetes a 28 días, pero para un plazo mayor.
- **SF43947**: `Cetes_182_dias` - Tasa de rendimiento de los Certificados de la Tesorería (Cetes) a 182 días. Representa el rendimiento de estos instrumentos para un plazo aún mayor.

## Indicadores económicos:
- **SP1**: `Inflacion_INPC` - Índice Nacional de Precios al Consumidor (INPC). Mide la variación de los precios de una canasta de bienes y servicios representativa del consumo de los hogares mexicanos. Es un indicador clave de la inflación.
- **SG29**: `PIB` - Producto Interno Bruto (PIB). Mide el valor total de los bienes y servicios producidos en un país durante un periodo específico. Es un indicador fundamental de la salud económica de un país.
- **SP68257**: `UDIS` - Unidades de Inversión. Son unidades de cuenta cuyo valor en pesos se actualiza diariamente con base en la inflación. Se utilizan en operaciones financieras para mantener el poder adquisitivo.
- **SP68264**: `IPC_Consumidor` - Índice de Precios al Consumidor. Similar al INPC, mide la variación de los precios al consumidor y es un indicador clave de la inflación.
- **SP68270**: `Salario_Minimo` - Salario mínimo. Es el salario mínimo legal que los empleadores deben pagar a sus trabajadores. Sirve como referencia para la política laboral y social.

## Precios de commodities:
- **SF60653**: `Oro` - Precio del oro. Mide el valor del oro en el mercado internacional. Es un indicador importante para los mercados financieros y de materias primas.
- **SF60654**: `Plata` - Precio de la plata. Similar al precio del oro, pero para la plata. Es relevante para la industria y los mercados financieros.
- **SF60655**: `Cobre` - Precio del cobre. Mide el valor del cobre en el mercado internacional. Es un indicador importante para la industria y la economía global.

## Índices bursátiles:
- **SP432**: `Dow_Jones` - Índice Dow Jones Industrial Average. Es un índice que representa el rendimiento de 30 grandes empresas cotizadas en bolsa en Estados Unidos. Es un indicador de la salud del mercado de valores estadounidense.
- **SP434**: `NASDAQ` - Índice NASDAQ Composite. Mide el rendimiento de más de 3,000 empresas cotizadas en la bolsa NASDAQ, muchas de las cuales son del sector tecnológico.
- **SP443**: `S&P_500` - Índice S&P 500. Mide el rendimiento de 500 empresas grandes cotizadas en bolsa en Estados Unidos. Es uno de los principales indicadores de la economía estadounidense.

In [17]:
import requests
import pandas as pd
import numpy as np

In [21]:
def descarga_bmx_serie(serie, fechainicio, fechafin, token):
    url = f'https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie}/datos/{fechainicio}/{fechafin}'
    headers = {'Bmx-Token': token}
    response = requests.get(url, headers=headers)
    status = response.status_code

    if status != 200:
        print(f'Error en la consulta de la serie {serie}, código {status}')
        return None

    raw_data = response.json()
    data = raw_data['bmx']['series'][0]['datos']
    df = pd.DataFrame(data)
    df['dato'] = df['dato'].apply(lambda x: float(x.replace(',', '')) if x not in ['N/E', ''] else None)
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df.set_index('fecha', inplace=True)
    return df

# Ejemplos de uso:
#dolares_bmx = descarga_bmx_serie('SF63528', '2016-01-01', '2023-12-31', token)
# tiie28_bmx = descarga_bmx_serie('SF43783', '2016-01-01', '2023-12-31', token)
# udis_bmx = descarga_bmx_serie('SP68257', '2016-01-01', '2023-12-31', token)
# cetes_bmx = descarga_bmx_serie('SF282', '2016-01-01', '2023-12-31', token)
salario_minimo = descarga_bmx_serie('SP68257', '2016-01-01', '2023-12-31', token)



In [ ]:
# Tu token de BANXICO
token = '211229616094fbb4b00e06d38c88b1d6d99c244d5b8f74e20dcb7663876ec1ca'

In [ ]:
# Diccionario de series y sus nombres descriptivos, serian los que se van a consultar en la API de BANXICO
series_info = {
    'SF63528': 'Dólares',
    'SF43718': 'TIIE_28_dias',
    'SP1': 'Inflacion_INPC',
    'SG29': 'PIB',
    'SP74625': 'Precio_petroleo',
    'SF43879': 'Indice_Bursatil_IPC'
}

In [ ]:
# Obtener los datos para todas las series en el diccionario
dataframes = {}
fechainicio = '2020-01-01'
fechafin = '2023-12-31'

In [ ]:
for serie, nombre in series_info.items():
    df = descarga_bmx_serie(serie, fechainicio, fechafin, token)
    if df is not None:
        df.columns = [nombre]  # Renombrar la columna 'dato' con el nombre descriptivo
        dataframes[nombre] = df

In [ ]:
# Mostrar los datos obtenidos (ejemplo: mostrar las primeras filas de cada DataFrame)
for nombre, df in dataframes.items():
    if df is not None:
        print(f"Datos de {nombre}:")
        print(df.head())
        print()

In [16]:
# Los DataFrames están ahora almacenados en el diccionario 'dataframes'
# Puedes acceder a cada DataFrame usando 'dataframes["nombre_de_la_serie"]'
# Ejemplo: df_tiie = dataframes["TIIE_28_dias"]

# Comentario sobre todas las posibles series que podrías extraer de la API de BANXICO
# Aquí hay algunos ejemplos adicionales de series que podrías extraer, con sus IDs:

# Tasas de interés:
# 'SF43936': 'TIIE_91_dias'
# 'SF43945': 'Cetes_28_dias'
# 'SF43946': 'Cetes_91_dias'
# 'SF43947': 'Cetes_182_dias'

# Indicadores económicos:
# 'SP68257': 'UDIS'
# 'SP68264': 'IPC_Consumidor'
# 'SP68270': 'Salario_Minimo'

# Precios de commodities:
# 'SF60653': 'Oro'
# 'SF60654': 'Plata'
# 'SF60655': 'Cobre'

# Índices bursátiles:
# 'SP432': 'Dow_Jones'
# 'SP434': 'NASDAQ'
# 'SP443': 'S&P_500'

# Ejemplo de cómo añadir más series a 'series_info':
# series_info = {
#     'SF63528': 'Dólares',
#     'SF43718': 'TIIE_28_dias',
#     'SP1': 'Inflacion_INPC',
#     'SG29': 'PIB',
#     'SP74625': 'Precio_petroleo',
#     'SF43879': 'Indice_Bursatil_IPC',
#     'SF43936': 'TIIE_91_dias',
#     'SF43945': 'Cetes_28_dias',
#     'SF43946': 'Cetes_91_dias',
#     'SF43947': 'Cetes_182_dias',
#     'SP68257': 'UDIS',
#     'SP68264': 'IPC_Consumidor',
#     'SP68270': 'Salario_Minimo',
#     'SF60653': 'Oro',
#     'SF60654': 'Plata',
#     'SF60655': 'Cobre',
#     'SP432': 'Dow_Jones',
#     'SP434': 'NASDAQ',
#     'SP443': 'S&P_500'
# }

# Puedes obtener una lista completa de series disponibles en la documentación de la API de BANXICO
# y actualizar el diccionario 'series_info' con las series que necesitas


Datos de Dólares:
            Dólares
fecha              
2020-01-02  18.8817
2020-01-03  18.8673
2020-01-06  18.8270
2020-01-07  18.8852
2020-01-08  18.7980

Datos de TIIE_28_dias:
            TIIE_28_dias
fecha                   
2020-01-02       18.8817
2020-01-03       18.8673
2020-01-06       18.8270
2020-01-07       18.8852
2020-01-08       18.7980

Datos de Inflacion_INPC:
            Inflacion_INPC
fecha                     
2020-01-01         106.447
2020-02-01         106.889
2020-03-01         106.838
2020-04-01         105.755
2020-05-01         106.162

Datos de PIB:
                  PIB
fecha                
2020-01-01   360259.7
2020-02-01   654102.8
2020-03-01  1074843.1
2020-04-01  1402819.8
2020-05-01  1738458.4

Datos de Precio_petroleo:
            Precio_petroleo
fecha                      
2020-01-01       105.577099
2020-02-01       105.961430
2020-03-01       106.272453
2020-04-01       106.653927
2020-05-01       106.972390

Datos de Indice_Bursatil_IPC:
     

In [25]:
df_cetes_28_dias = dataframes["Cetes_28_dias"]

KeyError: 'Cetes_28_dias'

In [31]:
import requests
import pandas as pd

# Tu token de BANXICO
token = '211229616094fbb4b00e06d38c88b1d6d99c244d5b8f74e20dcb7663876ec1ca'

# Diccionario de todas las series y sus nombres descriptivos
series_info = {
    'SF63528': 'Dólares',
    'SF43718': 'TIIE_28_dias',
    'SP1': 'Inflacion_INPC',
    'SG29': 'PIB',
    'SP74625': 'Precio_petroleo',
    'SF43879': 'Indice_Bursatil_IPC',
    'SF43936': 'TIIE_91_dias',
    'SF43945': 'Cetes_28_dias',
    'SF43946': 'Cetes_91_dias',
    'SF43947': 'Cetes_182_dias',
    'SP68257': 'UDIS',
    'SP68264': 'IPC_Consumidor',
    'SP68270': 'Salario_Minimo',
    'SF60653': 'Oro',
    'SF60654': 'Plata',
    'SF60655': 'Cobre',
    'SP432': 'Dow_Jones',
    'SP434': 'NASDAQ',
    'SP443': 'S&P_500'
}

def descarga_bmx_serie(serie, fechainicio, fechafin, token):
    url = f'https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie}/datos/{fechainicio}/{fechafin}'
    headers = {'Bmx-Token': token}
    response = requests.get(url, headers=headers)
    status = response.status_code

    if status != 200:
        print(f'Error en la consulta de la serie {serie}, código {status}')
        return None

    raw_data = response.json()
    # Verificar que la estructura de la respuesta es la esperada
    if 'bmx' not in raw_data or 'series' not in raw_data['bmx'] or len(raw_data['bmx']['series']) == 0 or 'datos' not in raw_data['bmx']['series'][0]:
        print(f'Estructura de la respuesta inesperada para la serie {serie}')
        print(raw_data)  # Imprimir la respuesta para depuración
        return None
    
    # Extraer los datos de la serie
    data = raw_data['bmx']['series'][0]['datos']
    df = pd.DataFrame(data)
    # Convertir valores a numéricos, manejando valores no disponibles
    df['dato'] = df['dato'].apply(lambda x: float(x.replace(',', '')) if x not in ['N/E', ''] else None)
    # Convertir las fechas a formato datetime
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df.set_index('fecha', inplace=True)
    return df

def obtener_series(series_ids, fechainicio, fechafin, token):
    dataframes = {}
    for serie in series_ids:
        nombre = series_info.get(serie, 'Desconocido')
        df = descarga_bmx_serie(serie, fechainicio, fechafin, token)
        if df is not None:
            df.columns = [nombre]  # Renombrar la columna 'dato' con el nombre descriptivo
            dataframes[nombre] = df
    return dataframes

# Definir las series que deseas descargar
series_a_descargar = [
    'SF43718',  # TIIE_28_dias
    'SF43936',  # TIIE_91_dias
    'SF43945',  # Cetes_28_dias
    'SP1',      # Inflacion_INPC
    'SG29',     # PIB
    'SP68257',  # UDIS
    'SF60653',  # Oro
    'SF60654',  # Plata
    'SF60655'   # Cobre
]

# Fechas de inicio y fin
fechainicio = '2020-01-01'
fechafin = '2023-12-31'

# Obtener los datos para las series especificadas
dataframes = obtener_series(series_a_descargar, fechainicio, fechafin, token)

# Mostrar los datos obtenidos (ejemplo: mostrar las primeras filas de cada DataFrame)
for nombre, df in dataframes.items():
    if df is not None:
        print(f"Datos de {nombre}:")
        print(df.head())
        print()

# Los DataFrames están ahora almacenados en el diccionario 'dataframes'
# Puedes acceder a cada DataFrame usando 'dataframes["nombre_de_la_serie"]'
# Ejemplo: df_tiie_28_dias = dataframes["TIIE_28_dias"]


Estructura de la respuesta inesperada para la serie SF60654
{'bmx': {'series': [{'idSerie': 'SF60654', 'titulo': ''}]}}
Estructura de la respuesta inesperada para la serie SF60655
{'bmx': {'series': [{'idSerie': 'SF60655', 'titulo': ''}]}}
Datos de TIIE_28_dias:
            TIIE_28_dias
fecha                   
2020-01-02       18.8817
2020-01-03       18.8673
2020-01-06       18.8270
2020-01-07       18.8852
2020-01-08       18.7980

Datos de TIIE_91_dias:
            TIIE_91_dias
fecha                   
2020-01-02          7.25
2020-01-09          7.26
2020-01-16          7.00
2020-01-23          7.05
2020-01-30          7.04

Datos de Cetes_28_dias:
            Cetes_28_dias
fecha                    
2020-01-02           7.18
2020-01-09            NaN
2020-01-16            NaN
2020-01-23            NaN
2020-01-30           6.97

Datos de Inflacion_INPC:
            Inflacion_INPC
fecha                     
2020-01-01         106.447
2020-02-01         106.889
2020-03-01         106